### Installing the necessities

!conda update conda
!conda update numpy

!pip install nltk
!pip install spacy
!pip install transformers
!pip install torch
!pip install requests
!pip install beautifulsoup4
!pip install python-dotenv
!python -m spacy download en_core_web_sm
!pip install langchain
!pip install -U langchain-community
!pip install openai==1.32.0 httpx==0.27.0
!pip install ipywidgets --upgrade


### Importing the libraries

In [1]:

import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'
os.environ['OMP_NUM_THREADS'] = '1'

import httpx#

import pickle

import nltk
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import wordpunct_tokenize 

from nltk.stem import SnowballStemmer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')


import spacy
from transformers import pipeline, set_seed

import torch
import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv

import langchain
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

import openai


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


/Users/adarshbhattarai/anaconda3/envs/dental_chatbot/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/adarshbhattarai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'''initial_url = [
    'https://www.mouthhealthy.org/all-topics-a-z/abscess',
    'https://www.mouthhealthy.org/all-topics-a-z/anesthesia-and-sedation-for-your-child',
    'https://publications.aap.org/pediatrics/article/143/6/e20191000/37173/Guidelines-for-Monitoring-and-Management-of?autologincheck=redirected',
    'https://www.mouthhealthy.org/all-topics-a-z/antibiotic-prophylaxis-for-heart-patients',
    'https://www.mouthhealthy.org/all-topics-a-z/joint-replacement',
    'https://www.mouthhealthy.org/all-topics-a-z/antibiotics-for-pain-and-swelling',
    'https://www.mouthhealthy.org/all-topics-a-z/anxiety',
    'https://www.mouthhealthy.org/dental-care-concerns/how-to-choose-a-dentist',
    'https://www.mouthhealthy.org/en/adamember',
    'https://www.mouthhealthy.org/all-topics-a-z/baby-teeth',
    'https://www.mouthhealthy.org/all-topics-a-z/sealants',
    'https://www.mouthhealthy.org/life-stages/babies-and-kids/playing-the-tooth-fairy',
    'https://www.mouthhealthy.org/-/media/project/ada-organization/ada/mouthhealthy/files/tooth-fairy/toothfairy_scrapbook_page_lost_tooth.pdf',
    'https://www.mouthhealthy.org/all-topics-a-z/bad-breath',
    'https://sjogrens.org/understanding-sjogrens/symptoms',
    'https://www.mouthhealthy.org/common-myths-of-gum-disease',
    'https://www.mouthhealthy.org/all-topics-a-z/plaque',
    'https://www.mouthhealthy.org/all-topics-a-z/smoking',
    'https://www.mouthhealthy.org/life-stages/pregnancy',
    'https://www.mouthhealthy.org/life-stages/pregnancy/pregnancy-dental-concerns',
    'https://www.mouthhealthy.org/life-stages/pregnancy/nutrition-pregnancy',
    'https://www.mouthhealthy.org/all-topics-a-z/gingivitis',
    'https://www.mouthhealthy.org/all-topics-a-z/bleeding-gums',
    'https://www.mouthhealthy.org/all-topics-a-z/flossing',
    'https://www.mouthhealthy.org/all-topics-a-z/breastfeeding',
    'https://www.mouthhealthy.org/all-topics-a-z/tooth-decay-with-baby-bottles',
    'https://www.mouthhealthy.org/all-topics-a-z/teeth-grinding',
    'https://www.mouthhealthy.org/all-topics-a-z/tmj',
    'https://www.ncbi.nlm.nih.gov/books/NBK501922/?report=classic',
    'https://www.mouthhealthy.org/all-topics-a-z/bridges',
    'https://www.mouthhealthy.org/all-topics-a-z/cancer-dental-health',
    'https://www.mouthhealthy.org/all-topics-a-z/oral-cancer',
    'https://www.mouthhealthy.org/all-topics-a-z/smoking-alternatives',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/guide-quitting-smoking/staying-tobacco-free-after-you-quit-smoking.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/guide-quitting-smoking/nicotine-replacement-therapy.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/guide-quitting-smoking/getting-help-with-the-mental-part-of-tobacco-addiction.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/helping-a-smoker-quit.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/guide-quitting-smoking/prescription-drugs-to-help-you-quit-smoking.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/guide-quitting-smoking/quitting-smoking-help-for-cravings-and-tough-situations.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/carcinogens-found-in-tobacco-products.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/health-risks-of-tobacco/smokeless-tobacco.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/e-cigarettes-vaping/what-do-we-know-about-e-cigarettes.html',
    'https://www.cancer.org/cancer/risk-prevention/tobacco/is-any-type-of-smoking-safe.html',
    'https://www.cancer.org/cancer/risk-prevention/diet-physical-activity.html',
    'https://www.cancer.org/cancer/risk-prevention/sun-and-uv.html',
    'https://www.cancer.org/cancer/risk-prevention/hpv.html',
    'https://www.cancer.org/cancer/risk-prevention/hpv/what-is-hpv.html',
    'https://www.cancer.org/cancer/risk-prevention/hpv/types-of-hpv.html',
    'https://www.cancer.org/cancer/risk-prevention/hpv/hpv-and-cancer-info.html',
    'https://www.cancer.org/cancer/risk-prevention/hpv/hpv-signs-and-symptoms.html',
    'https://www.cancer.org/cancer/risk-prevention/hpv/hpv-prevention.html',
    'https://www.cancer.org/cancer/risk-prevention/genetics.html',
    'https://www.cancer.org/cancer/risk-prevention/genetics/family-cancer-syndromes.html',
    'https://www.cancer.org/cancer/risk-prevention/genetics/genetic-testing-for-cancer-risk.html',
    'https://www.cancer.org/cancer/risk-prevention/radiation-exposure/x-rays-gamma-rays.html',
    'https://www.cancer.org/cancer/risk-prevention/radiation-exposure/x-rays-gamma-rays/what-are-xrays-and-gamma-rays.html',
    'https://www.cancer.org/cancer/risk-prevention/radiation-exposure/x-rays-gamma-rays/how-are-people-exposed.html',
    'https://www.cancer.org/cancer/risk-prevention/radiation-exposure/x-rays-gamma-rays/do-xrays-and-gamma-rays-cause-cancer.html',
    'https://www.mouthhealthy.org/all-topics-a-z/female-hormones-and-dental-health',
    'https://www.mouthhealthy.org/all-topics-a-z/fillings-gold-inlays',
    'https://www.mouthhealthy.org/all-topics-a-z/fillings-silver-colored',
    'https://www.mouthhealthy.org/all-topics-a-z/fillings-tooth-colored'
]
'''

In [2]:
"""url = 'https://www.mouthhealthy.org/en/az-topics/h/halitosis'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')
    paragraph_texts = [p.get_text() for p in paragraphs]

    if len(paragraph_texts) > 0:
        for text in paragraph_texts:
            print(text)
    else:
        print("No paragraphs found.")
else:
    print("Failed to fetch the webpage. Status code:", response.status_code)"""

'url = \'https://www.mouthhealthy.org/en/az-topics/h/halitosis\'\nheaders = {\n    \'User-Agent\': \'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36\'\n}\nresponse = requests.get(url, headers=headers)\n\nif response.status_code == 200:\n    soup = BeautifulSoup(response.content, \'html.parser\')\n    paragraphs = soup.find_all(\'p\')\n    paragraph_texts = [p.get_text() for p in paragraphs]\n\n    if len(paragraph_texts) > 0:\n        for text in paragraph_texts:\n            print(text)\n    else:\n        print("No paragraphs found.")\nelse:\n    print("Failed to fetch the webpage. Status code:", response.status_code)'

In [3]:


'''urls = list(set(initial_url))

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}


class Content:
    def __init__(self, url, paragraphs):
        self.url = url
        self.paragraphs = paragraphs

def read(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        return [p.get_text() for p in paragraphs]
    except requests.exceptions.HTTPError as errh:
        print("Http Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        print("OOps: Something Else", err)

contents = []

for url in urls:
    try:
        paragraphs = read(url)
        content = Content(url, paragraphs)
        contents.append(content)
        time.sleep(random.uniform(1, 3))
    except Exception as e:
        continue

# Verify the contents
for content in contents:
    print(f"URL: {content.url}")
    print(f"Paragraphs:")
    if content.paragraphs:
        for p in content.paragraphs:
            print(p)
    print("-" * 80)
'''

'urls = list(set(initial_url))\n\nheaders = {\n    \'User-Agent\': \'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3\'\n}\n\n\nclass Content:\n    def __init__(self, url, paragraphs):\n        self.url = url\n        self.paragraphs = paragraphs\n\ndef read(url):\n    try:\n        response = requests.get(url, headers=headers)\n        response.raise_for_status()  # Raise an exception for bad status codes\n        soup = BeautifulSoup(response.content, \'html.parser\')\n        paragraphs = soup.find_all(\'p\')\n        return [p.get_text() for p in paragraphs]\n    except requests.exceptions.HTTPError as errh:\n        print("Http Error:", errh)\n    except requests.exceptions.ConnectionError as errc:\n        print("Error Connecting:", errc)\n    except requests.exceptions.Timeout as errt:\n        print("Timeout Error:", errt)\n    except requests.exceptions.RequestException as err:\n        print("OOps: Something El

# Write to a pickle file to load it next time without having to call the API again

In [4]:
 
'''import pickle

# Write to pickle file
with open('output.pkl', 'wb') as f:
    pickle.dump(contents, f)

# To retrieve the contents object later
with open('output.pkl', 'rb') as f:
    contents = pickle.load(f)'''


"import pickle\n\n# Write to pickle file\nwith open('output.pkl', 'wb') as f:\n    pickle.dump(contents, f)\n\n# To retrieve the contents object later\nwith open('output.pkl', 'rb') as f:\n    contents = pickle.load(f)"

### PRACTICE BLOCK

In [5]:

'''class Content:
    def __init__(self, url, paragraphs):
        self.url = url
        self.paragraphs = paragraphs


urls = [
    'https://www.mouthhealthy.org/all-topics-a-z/abscess',
    'https://www.mouthhealthy.org/all-topics-a-z/anesthesia-and-sedation-for-your-child',
    'https://www.mayoclinic.org/drugs-supplements/anesthetic-local-topical-application-dental-buccal-mucosa-route-gargle/side-effects/drg-20069969'
]

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}

def read(url):
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for bad status codes
        soup = BeautifulSoup(response.content, 'html.parser')
        paragraphs = soup.find_all('p')
        return [p.get_text() for p in paragraphs]
    except requests.exceptions.HTTPError as errh:
        print("Http Error:", errh)
    except requests.exceptions.ConnectionError as errc:
        print("Error Connecting:", errc)
    except requests.exceptions.Timeout as errt:
        print("Timeout Error:", errt)
    except requests.exceptions.RequestException as err:
        print("OOps: Something Else", err)

contents = []

for url in urls:
    paragraphs = read(url)
    content = Content(url, paragraphs)
    contents.append(content)
    time.sleep(1)

# Save the contents object to a file
with open('outputt.pkl', 'wb') as f:
    pickle.dump(contents, f)

# To retrieve the contents object later
with open('outputt.pkl', 'rb') as f:
    contents = pickle.load(f)

# Now contents contains the same object as before
contents
'''

'class Content:\n    def __init__(self, url, paragraphs):\n        self.url = url\n        self.paragraphs = paragraphs\n\n\nurls = [\n    \'https://www.mouthhealthy.org/all-topics-a-z/abscess\',\n    \'https://www.mouthhealthy.org/all-topics-a-z/anesthesia-and-sedation-for-your-child\',\n    \'https://www.mayoclinic.org/drugs-supplements/anesthetic-local-topical-application-dental-buccal-mucosa-route-gargle/side-effects/drg-20069969\'\n]\n\nheaders = {\n    \'User-Agent\': \'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3\'\n}\n\ndef read(url):\n    try:\n        response = requests.get(url, headers=headers)\n        response.raise_for_status()  # Raise an exception for bad status codes\n        soup = BeautifulSoup(response.content, \'html.parser\')\n        paragraphs = soup.find_all(\'p\')\n        return [p.get_text() for p in paragraphs]\n    except requests.exceptions.HTTPError as errh:\n        print("Http Error

### Loading the pickle file

In [6]:

"""# Load the content from the pickle file

with open('allContent.pkl', 'rb') as f:
    all_content = pickle.load(f)

# Iterate over all_content and print each content's URL and paragraphs

for content in all_content:
    print(content.url, content.paragraphs)
"""

"# Load the content from the pickle file\n\nwith open('allContent.pkl', 'rb') as f:\n    all_content = pickle.load(f)\n\n# Iterate over all_content and print each content's URL and paragraphs\n\nfor content in all_content:\n    print(content.url, content.paragraphs)\n"

In [7]:

all_content=[]

class Content:
    def __init__(self, url, paragraphs):
        self.url = url
        self.paragraphs = paragraphs

with open('allContent.pkl', 'rb') as f:
    all_content = pickle.load(f)

all_content


### **************  END OF text preprocessing **************

## Tokenization - Paragraph - Sentence 

### Iterate over each Content object and tokenize the paragraphs

In [8]:
"""
for content in all_content:
    for paragraph in content.paragraphs:
        sentences = sent_tokenize(paragraph)

        print(sentences)
        """

'\nfor content in all_content:\n    for paragraph in content.paragraphs:\n        sentences = sent_tokenize(paragraph)\n\n        print(sentences)\n        '

## Tokenization - Sentence - Words

In [9]:
"""
# Flatten the list of paragraphs
all_paragraphs = [paragraph for content in all_content for paragraph in content.paragraphs]


# Tokenize all words from all paragraphs
all_words = [word_tokenize(paragraph) for paragraph in all_paragraphs]


# Print all tokenized words
for words in all_words:
    print(words)"""

'\n# Flatten the list of paragraphs\nall_paragraphs = [paragraph for content in all_content for paragraph in content.paragraphs]\n\n\n# Tokenize all words from all paragraphs\nall_words = [word_tokenize(paragraph) for paragraph in all_paragraphs]\n\n\n# Print all tokenized words\nfor words in all_words:\n    print(words)'

## Sentence & Word Tokenization

In [10]:

"""for content in all_content:
    for paragraph in content.paragraphs:
        sentences = sent_tokenize(paragraph)
        for sentence in sentences:
            words = word_tokenize(sentence)
            print(words)"""
            

'for content in all_content:\n    for paragraph in content.paragraphs:\n        sentences = sent_tokenize(paragraph)\n        for sentence in sentences:\n            words = word_tokenize(sentence)\n            print(words)'

### To treat punctuation as separate word

In [11]:
"""# Tokenize sentences and treat punctuation as separate tokens
for content in all_content:
    for paragraph in content.paragraphs:
        sentences = sent_tokenize(paragraph)
        for sentence in sentences:
            words = wordpunct_tokenize(sentence)
            print(words)"""

'# Tokenize sentences and treat punctuation as separate tokens\nfor content in all_content:\n    for paragraph in content.paragraphs:\n        sentences = sent_tokenize(paragraph)\n        for sentence in sentences:\n            words = wordpunct_tokenize(sentence)\n            print(words)'

#### Tokenize all words from all paragraphs

In [12]:
all_words = [wordpunct_tokenize(paragraph) for content in all_content for paragraph in content.paragraphs]

# Print all tokenized words

for words in all_words:
    print(words)


['ACS', '&', 'ASCO', 'are', 'Stronger', 'Together', ':', 'Cancer', '.', 'Net', 'content', 'is', 'now', 'available', 'on', 'cancer', '.', 'org', '.']
['Español']
['PDFs', 'by', 'language']
['Español']
['PDFs', 'by', 'language']
['Our', '24', '/', '7', 'cancer', 'helpline', 'provides', 'information', 'and', 'answers', 'for', 'people', 'dealing', 'with', 'cancer', '.', 'We', 'can', 'connect', 'you', 'with', 'trained', 'cancer', 'information', 'specialists', 'who', 'will', 'answer', 'questions', 'about', 'a', 'cancer', 'diagnosis', 'and', 'provide', 'guidance', 'and', 'a', 'compassionate', 'ear', '.']
[]
['Chat', 'live', 'online']
['Select', 'the', 'Live', 'Chat', 'button', 'at', 'the', 'bottom', 'of', 'the', 'page']
[]
['Schedule', 'a', 'Video', 'Chat']
['Face', 'to', 'face', 'support']
[]
['Call', 'us', 'at', '1', '-', '800', '-', '227', '-', '2345']
['Available', 'any', 'time', 'of', 'day', 'or', 'night']
['Our', 'highly', 'trained', 'specialists', 'are', 'available', '24', '/', '7', 'v

### STEMMING  ( process of removing prefix/suffix to find root word)

### This might produce some error. eg, increase. If there is problem statement like review classification program, email sopam/notspam
### stemming is preffered.

In [13]:

"""stemming = PorterStemmer()

for words in all_words:
    for word in words:
            print(word + " ----> " + stemming.stem(word))"""
        

'stemming = PorterStemmer()\n\nfor words in all_words:\n    for word in words:\n            print(word + " ----> " + stemming.stem(word))'

In [14]:
### SnowballStemmer (Better than PorterStemmer)

In [15]:

"""
stemming = SnowballStemmer("english")

for words in all_words:
    for word in words:
        print(word + " ----> " + stemming.stem(word))"""


'\nstemming = SnowballStemmer("english")\n\nfor words in all_words:\n    for word in words:\n        print(word + " ----> " + stemming.stem(word))'

### Lemmatization ( it has dictionary for all the root word so moree accurate and while creating a chatbot, Q&A, text summarization)

In [16]:

lemmatizer = WordNetLemmatizer()

for words in all_words:
    for word in words:
        print(word + " ----> " + lemmatizer.lemmatize(word,pos = "v"))


ACS ----> ACS
& ----> &
ASCO ----> ASCO
are ----> be
Stronger ----> Stronger
Together ----> Together
: ----> :
Cancer ----> Cancer
. ----> .
Net ----> Net
content ----> content
is ----> be
now ----> now
available ----> available
on ----> on
cancer ----> cancer
. ----> .
org ----> org
. ----> .
Español ----> Español
PDFs ----> PDFs
by ----> by
language ----> language
Español ----> Español
PDFs ----> PDFs
by ----> by
language ----> language
Our ----> Our
24 ----> 24
/ ----> /
7 ----> 7
cancer ----> cancer
helpline ----> helpline
provides ----> provide
information ----> information
and ----> and
answers ----> answer
for ----> for
people ----> people
dealing ----> deal
with ----> with
cancer ----> cancer
. ----> .
We ----> We
can ----> can
connect ----> connect
you ----> you
with ----> with
trained ----> train
cancer ----> cancer
information ----> information
specialists ----> specialists
who ----> who
will ----> will
answer ----> answer
questions ----> question
about ----> about
a ----> a

### STOPWORDS


### Retrieve the list of English stopwords



In [17]:

stop_words = (stopwords.words('english'))
print(stop_words)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '


### Initializing the stemmer and stopwords


In [23]:
stemmer = PorterStemmer()

stemmed_sentences = []

for content in all_content:
    for paragraph in content.paragraphs:
        
        sentences = sent_tokenize(paragraph)
        for sentence in sentences:
            stemmed_sentence = stemmer.stem(sentence)
            stemmed_sentences.append(stemmed_sentence)
            print(stemmed_sentence)

acs & asco are stronger together: cancer.net content is now available on cancer.org.
español
pdfs by languag
español
pdfs by languag
our 24/7 cancer helpline provides information and answers for people dealing with cancer.
we can connect you with trained cancer information specialists who will answer questions about a cancer diagnosis and provide guidance and a compassionate ear.
chat live onlin
select the live chat button at the bottom of the pag
schedule a video chat
face to face support
call us at 1-800-227-2345
available any time of day or night
our highly trained specialists are available 24/7 via phone and on weekdays can assist through video calls and online chat.
we connect patients, caregivers, and family members with essential services and resources at every step of their cancer journey.
ask us how you can get involved and support the fight against cancer.
some of the topics we can assist with include:
for medical questions, we encourage you to review our information with you

In [ ]:
type (stemmed_sentences)

### Apply Stopwords and Filter And then Apply STEMMING

In [24]:
"""stemmer = PorterStemmer()

for i in range(len(stemmed_sentences)):
    words = nltk.word_tokenize(stemmed_sentences[i])
    words = [stemmer.stem(word) for word in words if word not in stop_words]
    stemmed_sentences[i] = " ".join(words) 

stemmed_sentences"""

['ac & asco stronger togeth : cancer.net content avail cancer.org .',
 'español',
 'pdf languag',
 'español',
 'pdf languag',
 '24/7 cancer helplin provid inform answer peopl deal cancer .',
 'connect train cancer inform specialist answer question cancer diagnosi provid guidanc compassion ear .',
 'chat live onlin',
 'select live chat button bottom pag',
 'schedul video chat',
 'face face support',
 'call us 1-800-227-2345',
 'avail time day night',
 'highli train specialist avail 24/7 via phone weekday assist video call onlin chat .',
 'connect patient , caregiv , famili member essenti servic resourc everi step cancer journey .',
 'ask us get involv support fight cancer .',
 'topic assist includ :',
 'medic question , encourag review inform doctor .',
 'cancer risk prevent',
 'e-cigarett known mani differ name , sometim peopl find hard understand realli known devic .',
 'address common question peopl ask e-cigarett .',
 'e-cigarett known mani differ name , includ e-cig , electron nico

In [26]:
"""stemming = SnowballStemmer("english")
stop_words = set(stopwords.words('english'))

for i in range(len(stemmed_sentences)):
    words = word_tokenize(stemmed_sentences[i])
    words = [stemming.stem(word) for word in words if word.lower() not in stop_words]
    stemmed_sentences[i] = " ".join(words) 

stemmed_sentences"""

'stemming = SnowballStemmer("english")\nstop_words = set(stopwords.words(\'english\'))\n\nfor i in range(len(stemmed_sentences)):\n    words = word_tokenize(stemmed_sentences[i])\n    words = [stemming.stem(word) for word in words if word.lower() not in stop_words]\n    stemmed_sentences[i] = " ".join(words) \n\nstemmed_sentences'

### Using Lemmatization and snowballStemming

In [35]:
lemmatizer = WordNetLemmatizer()

for content in all_content:
    for paragraph in content.paragraphs:
        
        sentences = sent_tokenize(paragraph)
        for sentence in sentences:
            stemmed_sentence = stemmer.stem(sentence)
            stemmed_sentences.append(stemmed_sentence)
            print(stemmed_sentence)


acs & asco are stronger together: cancer.net content is now available on cancer.org.
español
pdfs by languag
español
pdfs by languag
our 24/7 cancer helpline provides information and answers for people dealing with cancer.
we can connect you with trained cancer information specialists who will answer questions about a cancer diagnosis and provide guidance and a compassionate ear.
chat live onlin
select the live chat button at the bottom of the pag
schedule a video chat
face to face support
call us at 1-800-227-2345
available any time of day or night
our highly trained specialists are available 24/7 via phone and on weekdays can assist through video calls and online chat.
we connect patients, caregivers, and family members with essential services and resources at every step of their cancer journey.
ask us how you can get involved and support the fight against cancer.
some of the topics we can assist with include:
for medical questions, we encourage you to review our information with you

In [36]:
for i in range(len(stemmed_sentences)):
    words = word_tokenize(stemmed_sentences[i])
    words = [word.lower() for word in words]  
    words = [lemmatizer.lemmatize(word, pos='v') for word in words if word.lower() not in stop_words]
    stemmed_sentences[i] = " ".join(words)
stemmed_sentences

['ac & asco stronger togeth : cancer.net content avail cancer.org .',
 'español',
 'pdf languag',
 'español',
 'pdf languag',
 '24/7 cancer helplin provid inform answer peopl deal cancer .',
 'connect train cancer inform specialist answer question cancer diagnosi provid guidanc compass ear .',
 'chat live onlin',
 'select live chat button bottom pag',
 'schedul video chat',
 'face face support',
 'call us 1-800-227-2345',
 'avail time day night',
 'high train specialist avail 24/7 via phone weekday assist video call onlin chat .',
 'connect patient , caregiv , famili member essenti servic resourc everi step cancer journey .',
 'ask us get involv support fight cancer .',
 'topic assist includ :',
 'medic question , encourag review inform doctor .',
 'cancer risk prevent',
 'e-cigarett know mani differ name , sometim peopl find hard understand realli know devic .',
 'address common question peopl ask e-cigarett .',
 'e-cigarett know mani differ name , includ e-cig , electron nicotin deli

### Finding Pos tag

In [45]:
"""import nltk
nltk.download('averaged_perceptron_tagger')
for i in range(len(stemmed_sentences)):
    words = word_tokenize(stemmed_sentences[i])
    words = [stemming.stem(word) for word in words if word.lower() not in stop_words]
    pos_tag = nltk.pos_tag(words)
    print(pos_tag)"""


"import nltk\nnltk.download('averaged_perceptron_tagger')\nfor i in range(len(stemmed_sentences)):\n    words = word_tokenize(stemmed_sentences[i])\n    words = [stemming.stem(word) for word in words if word.lower() not in stop_words]\n    pos_tag = nltk.pos_tag(words)\n    print(pos_tag)"

### ONE HOT ENCODING

### Advantage - easy to implement with Pythom

### Disadvantages
#### 1) Sparse matrix -- leads to overfitting 
#### 2) For ML, we need fixed size input, but one hot encoding maynot have it always
#### 3) No semantic meaning is getting captured
#### 4) Out of Vocabulary(OOV)

### Bag Of Words - (helps to do basic sentiment/ text classification, spam/notspam, etc)
#### We start by converting all into lower() asn removing the stop words

### Types
### 1) Binary bag of Wrords : Even though the count is 2, it forces the value to be 1 or 0.
### 2) Bag of Words : In normal BOW, we can increase the count to 2 or more, based on it's frequency.

### Advantages:
#### 1) Simple and Intuitive.
#### 2) Fixed size input that can be used in ML algorithms.

### Disdvantages: 
#### 1) Sparse matrix or Array -- leads to overfitting 
#### 2) Ordering of the word gets changed.
#### 3) No semantic meaning is getting captured
#### 4) Out of Vocabulary(OOV)


### TF- IDF [ Term Frequency And Inverse Document Frequency]

#### TF = No. of repitition of words in a sentence/No. of epitition of words in a sentence/

#### IDF = log base e (No. of sentences / No. of sentences containing the words)

### Advantages:
#### 1) Simple and Intuitive.
#### 2) Fixed size (Vocabulary size) input that can be used in ML algorithms.
#### 3) Word Importance is getting captured


### Disdvantages: 
#### 1) Sparse matrix or Array -- leads to overfitting 
#### 2) Out of Vocabulary(OOV)


### WORD EMBEDDINGS

#### TYPES BASED ON:
#### 1) Count Or Frequency ( OHE, BOW, TF-IDF )
#### 2) Deep Learning Trained Model( Word2Vec which has 2 types - i) Continuous BOW & ii) Skipgram)

###  Word2Vec

In [ ]:
### **************  END OF text preprocessing **************